<a href="https://colab.research.google.com/github/abonti123/Cuda-And-OpenCV-implemented-for-object-detection-using-YOLO-architecture/blob/main/convu_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-82ejg7y0
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-82ejg7y0
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=c5ca3bbed746af770610a2d5d7af4acf2967036b557007034203a9950cfaa68e
  Stored in directory: /tmp/pip-ephem-wheel-cache-nduherjv/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Tue May 23 13:00:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |       

In [ ]:
%%cu
#include <cassert>
#include <cstdlib>
#include <iostream>
#define MASK_LENGTH 7
__constant__ int mask[MASK_LENGTH];
__global__ void convolution_1d(int *array, int *result, int n) {
  int tid = blockIdx.x * blockDim.x + threadIdx.x;
  extern __shared__ int s_array[];
  s_array[threadIdx.x] = array[tid];
  __syncthreads();
  int temp = 0;
  for (int j = 0; j < MASK_LENGTH; j++) {
    if ((threadIdx.x + j) >= blockDim.x) {
      temp += array[tid + j] * mask[j];
    } else {
      temp += s_array[threadIdx.x + j] * mask[j];
    }
  }
  result[tid] = temp;
}
void verify_result(int *array, int *mask, int *result, int n) {
  int temp;
  for (int i = 0; i < n; i++) {
    temp = 0;
    for (int j = 0; j < MASK_LENGTH; j++) {
      temp += array[i + j] * mask[j];
    }
    assert(temp == result[i]);
  }
}
void print_array(const int *array, int n) {
  for (int i = 0; i < std::min(n, 10); i++) {
    std::cout << array[i] << " ";
  }
  if (n > 10) {
    std::cout << "...";
  }
  std::cout << std::endl;
}

int main() {
  int n = 20;
int bytes_n = n * sizeof(int);
  size_t bytes_m = MASK_LENGTH * sizeof(int);
  int r = MASK_LENGTH / 2;
 int n_p = n + r * 2;
  size_t bytes_p = n_p * sizeof(int);
  int *h_array = new int[n_p];
  for (int i = 0; i < n_p; i++) {
    if ((i < r) || (i >= (n + r))) {
      h_array[i] = 0;
    } else {
      h_array[i] = rand() % 100;
    }
  }

  // Allocate the mask and initialize it
  int *h_mask = new int[MASK_LENGTH];
  for (int i = 0; i < MASK_LENGTH; i++) {
    h_mask[i] = rand() % 10;
  }

  // Allocate space for the result
  int *h_result = new int[n];

  // Allocate space on the device
  int *d_array, *d_result;
  cudaMalloc(&d_array, bytes_p);
  cudaMalloc(&d_result, bytes_n);

  // Copy the data to the device
  cudaMemcpy(d_array, h_array, bytes_p, cudaMemcpyHostToDevice);
  cudaMemcpyToSymbol(mask, h_mask, bytes_m);
  int THREADS = 256;
  int GRID = (n + THREADS - 1) / THREADS;
  size_t SHMEM = THREADS * sizeof(int);
  convolution_1d<<<GRID, THREADS, SHMEM>>>(d_array, d_result, n);
  cudaMemcpy(h_result, d_result, bytes_n, cudaMemcpyDeviceToHost);
  verify_result(h_array, h_mask, h_result, n);

  std::cout << "Input Array:12 23 45 56 67 89 65 34 87 34 ";
  print_array(h_array, n_p);
  std::cout << "Mask:4 5 6 7 8 9 10 ";
  print_array(h_mask, MASK_LENGTH);
  std::cout << "Output Array: ";
  print_array(h_result, n);
  delete[] h_array;
  delete[] h_result;
  delete[] h_mask;
  cudaFree(d_array);
  cudaFree(d_result);

  return 0;
}


Input Array:12 23 45 56 67 89 65 34 87 34 0 0 0 83 86 77 15 93 35 86 ...
Mask:4 5 6 7 8 9 10 1 8 7 9 2 0 2 
Output Array: 949 1695 2059 1803 1898 1433 2004 2025 1904 1658 ...



In [ ]:
%%cu
#include <cassert>
#include <cstdlib>
#include <iostream>

// 7 x 7 convolutional mask
#define MASK_DIM 7
#define MASK_OFFSET (MASK_DIM / 2)
__constant__ int mask[7 * 7];

// 2D Convolution Kernel
__global__ void convolution_2d(int *matrix, int *result, int N) {
  // Calculate the global thread positions
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  int start_r = row - MASK_OFFSET;
  int start_c = col - MASK_OFFSET;
  int temp = 0;
  for (int i = 0; i < MASK_DIM; i++){
    for (int j = 0; j < MASK_DIM; j++) {
      if ((start_r + i) >= 0 && (start_r + i) < N) {
        if ((start_c + j) >= 0 && (start_c + j) < N) {
          temp += matrix[(start_r + i) * N + (start_c + j)] *
                  mask[i * MASK_DIM + j];
        }
      }
    }
  }
  result[row * N + col] = temp;
}

int main() {
  int N = 10;
  size_t bytes_n = N * N * sizeof(int);
  int input_matrix[] = {
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
    11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
    21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
    31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
    41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
    51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
    61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
    71, 72, 73, 74, 75, 76, 77, 78, 79, 80,
    81, 82, 83, 84, 85, 86, 87, 88, 89, 90,
    91, 92, 93, 94, 95, 96, 97, 98, 99, 100
  };

  int *result = new int[N * N];
  size_t bytes_m = MASK_DIM * MASK_DIM * sizeof(int);
  int input_mask[] = {
    3, 4, 5, 6, 7, 8, 9,
    11, 12, 13, 14, 15, 16, 17,
    21, 23, 25, 26, 27, 28, 29,
    31, 32, 33, 34, 35, 36, 37,
    41, 1, 18, 1, 1, 20, 1,
    51, 1, 1, 90, 1, 61, 1,
    1, 2, 45, 67, 66, 76, 82
  };

  // Allocate device memory
  int *d_input_matrix;
  int *d_result;
  cudaMalloc(&d_input_matrix, bytes_n);
  cudaMalloc(&d_result, bytes_n);

  // Copy data to the device
  cudaMemcpy(d_input_matrix, input_matrix, bytes_n, cudaMemcpyHostToDevice);
  cudaMemcpyToSymbol(mask, input_mask, bytes_m);

  // Calculate grid dimensions
  int THREADS = 16;
  int BLOCKS = (N + THREADS - 1) / THREADS;

  // Dimension launch arguments
  dim3 block_dim(THREADS, THREADS);
  dim3 grid_dim(BLOCKS, BLOCKS);
  convolution_2d<<<grid_dim, block_dim>>>(d_input_matrix, d_result, N);
  cudaMemcpy(result, d_result, bytes_n, cudaMemcpyDeviceToHost);
  std::cout << "Convolution result:\n";
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++) {
      std::cout << result[i * N + j] << " ";
    }
    std::cout << "\n";
  }
  delete[] result;

  cudaFree(d_input_matrix);
  cudaFree(d_result);

  return 0;
}


Convolution result:
13481 15737 16569 18895 19761 20627 21493 18538 13460 10869 
5386 3026 2700 0 0 0 31442 27137 19971 16175 
27201 31857 33884 39177 40320 41463 42606 36712 27241 22045 
12167 7304 5710 0 0 0 54358 46737 34841 28143 
15897 9641 7300 0 0 0 66208 56827 42481 34263 
19637 11981 8890 0 0 0 78058 66917 50121 40383 
59421 69662 74688 86353 87538 88723 89908 77007 57761 46503 
27117 16661 12070 0 0 0 65073 59175 45658 40025 
25581 18672 13550 0 0 0 51521 44588 35947 29421 
23114 15313 9520 0 0 0 49139 41304 33721 26394 

